In [1]:
#importação das bibliotecas
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import os

from groq import Groq

#Requisição da API
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)
#iniciando chat com o prompt: Explain the importance of fast language models.
print(chat_completion.choices[0].message.content)

Fast language models are crucial in the field of natural language processing (NLP) due to their ability to efficiently process and generate human-like language. The importance of fast language models can be highlighted in the following areas:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems. These models can quickly understand and respond to user input, providing an seamless and interactive experience.
2. **Improved User Experience**: Fast language models can quickly generate text, summaries, or responses, reducing the time users need to wait for results. This leads to a better user experience, increased engagement, and higher satisfaction rates.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them ideal for large-scale applications such as customer service platforms, social media monitoring, and content generation.
4. **Reduced La

In [4]:
#Criando a classe Agent com as entradas client e system(prompt do sistema)
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []  #Esse campo será o histórico do chat. Nele as mensagens enviadas serão armazenadas, assim como os pensamentos do modelo.
        if self.system is not None:
            self.messages.append({"role": "system", "content": system})
    #Essa função permite que a instância da classe seja tradada como uma função. A intância poderá ser chamada com o argumento message, que é o prompt do usuário
    def __call__(self, message=""):
        if message:   #se houver message adicionar ele ao histórico
            self.messages.append({"role":"user", "content": message})
        result = self.execute()  #executar a função execute, que irá iniciar o chat
        self.messages.append({"role": "assistant", "content": result}) # adicionar o resultado às mensagens
        return result
    
    def execute(self): 
        completion = client.chat.completions.create( #criando o chat com o modelo llama3 e iniciando ele com as mensagens presentes no self.messages
            model = "llama3-70b-8192",
            messages = self.messages,
        )
        return completion.choices[0].message.content

In [5]:
#prompt do sistema
#definindo o fluxo de funcionamento do agente: pensamento, ação, pausa, observação e resposta(caso haja)
#nesse prompt nós apresentamos ao agente as ferramentas que lhe estão disponíveis.
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [6]:
# Ferramentas (Tools) 
def calculate(operation): #calculadora
    return eval(operation)

def get_planet_mass(planet) -> float: #encontrar a massa do planeta. Se o argumento corresponder a alguma string a função retornará o valor da string em massa
    match planet.lower():
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [7]:
neil_tyson = Agent(client, system = system_prompt)       #Instância da classe

In [8]:
result = neil_tyson("what is the mass of Earth times 5?") #Primeiro prompt
print(result)

Thought: I need to find the mass of Earth


In [10]:
result = neil_tyson()
print(result)

 
Action: get_planet_mass: Earth 
PAUSE


In [13]:
result = get_planet_mass("Earth")   #executando a função para que o agente receba o valor dela como observação
print(result)

5.972e+24


In [14]:
next_prompt = f"Observation: {result}"   #enviando a observação 
result = neil_tyson(next_prompt) # código seguirá a execução das ultimmas células até que a resposta seja obtida.
print(result)

Thought: I need to multiply this by 5


In [15]:
result = neil_tyson(next_prompt) # código seguirá a execução das ultimmas células até que a resposta seja obtida.
print(result)

Action: calculate: 5.972e24 * 5
PAUSE


In [16]:
result = calculate("5.972e24 * 5")
print(result)

2.9860000000000004e+25


In [17]:
next_prompt = f"Observation: {result}"   #enviando a observação 
next_prompt

'Observation: 2.9860000000000004e+25'

In [18]:
result = neil_tyson(next_prompt)
print(result)

Answer: The mass of Earth times 5 is 2.9860000000000004e+25.


In [19]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [20]:
#importando a biblioteca re para a identificação da ação e do argumento
import re
#agora a ideia é produzir um agente que funcione em loop, descartando a necessidade de ficar executando células manualmente
def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    while i < max_iterations: #define que o loop será executado até que i ( o contador ) seja menor que o número máximo de iterações
        i += 1 #incrimenta 1 a i após cada iteração
        result = agent(next_prompt) #Após cada loop result assume um valor diferente. No primeiro loop result está definido como query
        print(result)
        #Se pause e action estiver no resultado a biblioteca re irá achar qual é a ação e qual é o argumento da ação. 
        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0] # ferramenta encontrada
            arg = action[0][1] #argumento do ferramenta

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')") #aqui a "observação" será salva e adicionada ao próximo prompt
                next_prompt = f"Observation: {result_tool}"

            else:   #caso não haja ferramenta haverá um erro
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue #continua para a próxima iteração

        if "Answer" in result: #se o termo answer estiver nos resultados o loop é encerrado e a resposta estará armazenada em result
            break



In [21]:
agent_loop(10, system = system_prompt, query="what is the mass of the Earth plus the mass of Mercury, and All of it times 5")

Thought: I need to find the masses of Earth and Mercury, then add them together and multiply by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, I need to find the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: Now I have the masses of Earth and Mercury, I need to add them together.
Action: calculate: 5.972e24 + 3.285e23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now I have the sum of the masses of Earth and Mercury, I need to multiply it by 5.
Action: calculate: 6.300500000000001e24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the result of the multiplication, which is the answer to the question.

Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.1502500000000004e+25.
